In [1]:
import numpy as np
import math
import GPy
import matplotlib.pyplot as plt
import seaborn

In [5]:
import numpy as np
from scipy.stats import qmc

# Goldstein-Price function (as defined in previous responses)
def goldstein_price(data):
    x = data[0]
    y = data[1]
    val1 = (1 + (x+y+1)**2 * (19-14*x+3*x**2-14*y+6*x*y+3*y**2))
    val2 = (30 + ((2*x-3*y)**2) * (18-32*x+12*x**2+48*y-36*x*y+27*y**2))
    return val1 * val2


d = 2 

# Number of samples
n_samples = 32

# Create a Sobol sequence generator
sobol_engine = qmc.Sobol(d=d, scramble=False)  # scramble=True for better properties in high dimensions

# Generate Sobol points
sobol_points = sobol_engine.random_base2(m=int(np.log2(n_samples)))


# If n_samples is not a power of 2: (important!)
if not (n_samples != 0 and (n_samples & (n_samples - 1) == 0)): # Check if n_samples is not a power of 2
    sobol_points = sobol_engine.random(n=n_samples) #use random if not power of two


# Scale and shift to the Goldstein-Price domain [-2, 2]
domain_min = -2
domain_max = 2
scaled_sobol_points = domain_min + (domain_max - domain_min) * sobol_points
# Evaluate the function
function_values = np.array([goldstein_price(point) for point in scaled_sobol_points])

In [6]:
def base_kernel_search(X, y):
    kernel_comp = [GPy.kern.RBF(input_dim=X.shape[1], variance=1.0, lengthscale=1.0), 
                         GPy.kern.RatQuad(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Matern32(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Matern52(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.StdPeriodic(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Exponential(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0)]
    
    best_model = None
    best_bic = float('inf')
    mean = GPy.mappings.Constant(input_dim = X.shape[1], output_dim = 1, value = np.mean(y))


    for kernel1 in kernel_comp:
        kernel = kernel1
        model = GPy.models.GPRegression(X, y, kernel, noise_var=0.001**2, mean_function=mean)
        model.Gaussian_noise.variance.fix()
        model.optimize_restarts(10, verbose=False)
        n = model.num_data
        k = len(model.parameters)
        log_likelihood = model.log_likelihood()
        
        bic = k * np.log(n) - 2 * log_likelihood
        print(bic)
        print(model)

        if bic < best_bic:
            best_model = (kernel, model)
            best_bic = bic

    print(best_bic, best_model)

In [10]:
import warnings
with warnings.catch_warnings(action="ignore"):
    base_kernel_search(scaled_sobol_points, function_values.reshape(-1, 1))

LinAlgError: not positive definite, even with jitter.

In [ ]:
def kernel_search(X, y):
    kernel_comp = [GPy.kern.RBF(input_dim=X.shape[1], variance=1.0, lengthscale=1.0), 
                         GPy.kern.RatQuad(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Matern32(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Matern52(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.StdPeriodic(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Exponential(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0)]
    
    best_model = None
    best_bic = float('inf')
    mean_comp = [GPy.mappings.Constant(input_dim = X.shape[1], output_dim = 1, value = np.mean(y)), 
            GPy.mappings.Linear(input_dim = X.shape[1], output_dim = 1)]


    for kernel1 in kernel_comp:
        for kernel2 in kernel_comp:
            for mean in mean_comp:
                # Combine kernels
                kernel = kernel1 + kernel2
                model = GPy.models.GPRegression(X, y, kernel, noise_var=0.001**2, mean_function=mean)
                model.Gaussian_noise.variance.fix()
                model.optimize_restarts(10, verbose=False)
                n = model.num_data
                k = len(model.optimizer_array)
                log_likelihood = model.log_likelihood()
                
                bic = k * np.log(n) - 2 * log_likelihood
                print(bic)
                print(model)

                if bic < best_bic:
                    best_model = (kernel, model)
                    best_bic = bic

    print(best_bic, best_model)

In [ ]:
import warnings
with warnings.catch_warnings(action="ignore"):
    kernel_search(scaled_sobol_points, function_values.reshape(-1, 1))

In [89]:
def best_kernel_search(X, y):
    
    kernel = GPy.kern.StdPeriodic(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0) + GPy.kern.RBF(input_dim=X.shape[1], variance=1.0, lengthscale=1.0)
    mean = GPy.mappings.Constant(input_dim = X.shape[1], output_dim = 1, value = np.mean(y))
    model = GPy.models.GPRegression(X, y, kernel, noise_var=0.001**2, mean_function=mean)
    model.Gaussian_noise.variance.fix()
    model.optimize_restarts(100, verbose=False)

    n = model.num_data
    k = len(model.optimizer_array)
    log_likelihood = model.log_likelihood()
    
    bic = k * np.log(n) - 2 * log_likelihood
    print(bic)
    print(model)

    return model

In [91]:
store = best_kernel_search(scaled_sobol_points, function_values.reshape(-1, 1))

 /Users/anthonychiang/.pyenv/versions/3.11.5/lib/python3.11/site-packages/GPy/kern/src/standard_periodic.py:131: RuntimeWarning:overflow encountered in square
 /Users/anthonychiang/.pyenv/versions/3.11.5/lib/python3.11/site-packages/GPy/kern/src/standard_periodic.py:509: RuntimeWarning:overflow encountered in square
 /Users/anthonychiang/.pyenv/versions/3.11.5/lib/python3.11/site-packages/GPy/kern/src/standard_periodic.py:511: RuntimeWarning:divide by zero encountered in divide
 /Users/anthonychiang/.pyenv/versions/3.11.5/lib/python3.11/site-packages/GPy/kern/src/standard_periodic.py:511: RuntimeWarning:invalid value encountered in multiply
 /Users/anthonychiang/.pyenv/versions/3.11.5/lib/python3.11/site-packages/GPy/kern/src/standard_periodic.py:513: RuntimeWarning:divide by zero encountered in divide
 /Users/anthonychiang/.pyenv/versions/3.11.5/lib/python3.11/site-packages/GPy/kern/src/standard_periodic.py:513: RuntimeWarning:invalid value encountered in divide
 /Users/anthonychiang/

73.19605043468091

Name : GP regression
Objective : 26.200817508941277
Number of Parameters : 7
Number of Optimization Parameters : 6
Updates : True
Parameters:
  GP_regression.                |                value  |  constraints  |  priors
  constmap.C                    |   4.6311931729516935  |               |        
  sum.std_periodic.variance     |   0.2750368006713868  |      +ve      |        
  sum.std_periodic.period       |  0.19804650103615534  |      +ve      |        
  sum.std_periodic.lengthscale  |  0.39405807496177914  |      +ve      |        
  sum.rbf.variance              |   1.1217928115868112  |      +ve      |        
  sum.rbf.lengthscale           |   1.3178881298142384  |      +ve      |        
  Gaussian_noise.variance       |                1e-06  |   +ve fixed   |        


In [ ]:
prob_imp = np.min(function_values)


1.5143817062113494
